In [34]:
import numpy as np

a = 2.46 # angstrom
theta  = 1.00*np.pi/180 # degree to rad

# primitive vectors of SLG by Moon Koshino (2013)
lat_vec1 = a*np.array([1,0])
lat_vec2 = a*np.array([0.5,0.5*np.sqrt(3)])

N = 100

def Rotation(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array(((c, -s), (s, c)))

lat_vec_bottom1 = np.matmul(Rotation(-0.5*theta), lat_vec1)
lat_vec_bottom2 = np.matmul(Rotation(-0.5*theta), lat_vec2)
lat_vec_top1 = np.matmul(Rotation(0.5*theta), lat_vec1)
lat_vec_top2 = np.matmul(Rotation(0.5*theta), lat_vec2)

d0 = 3.35 # angstrom
a0 = 1.42 # angstrom
tau = a0*np.array([0,1]) # shift from A to B

def DistBottomTop(bottom_atom, sublattice_bottom, top_atom, sublattice_top):

    # bottom_atom and top_atom are indices [i,j] and [n,k] of lattice elements
    # sublattice =0 stands for A, =1 stands for B
    # returns distance between given atoms
    
    vec_bottom_atom = bottom_atom[0]*lat_vec_bottom1+bottom_atom[1]*lat_vec_bottom2
    vec_top_atom = top_atom[0]*lat_vec_top1+top_atom[1]*lat_vec_top2
    if sublattice_bottom == sublattice_top:
        return np.sqrt(np.linalg.norm(vec_bottom_atom-vec_top_atom)**2+d0**2)
    elif sublattice_bottom == 0:
        return np.sqrt(np.linalg.norm(vec_bottom_atom-vec_top_atom-tau)**2+d0**2)
    elif sublattice_bottom == 1:
        return np.sqrt(np.linalg.norm(vec_bottom_atom-vec_top_atom+tau)**2+d0**2)
    

#print(DistBottomTop([0,0],1,[0,5],1))

def ExtractFirsts(lst):
    return [item[0] for item in lst]



def ClosestAtomFromTop(bottom_atom, sublattice_bottom, sublattice_top):
    
    # finds [n,k] of atom from top layer within given sublattice
    # which is the closest to given [i,j] atom from sublattice_bottom from the bottom
    # returns distance to closest atom, its coordinates and vector in plane which connects bottom atom to top atom
    
    vec_bottom_atom = bottom_atom[0]*lat_vec_bottom1+bottom_atom[1]*lat_vec_bottom2 + 0.5*(2*sublattice_bottom-1)*tau
    dists = []
    window = int(1.1*theta*N) # a window which helps to shorten the area of searching for closest atom
    for n in range(bottom_atom[0]-window, bottom_atom[0]+window):
        for k in range(bottom_atom[1]-window, bottom_atom[1]+window):
            vec_top_atom = n*lat_vec_top1+k*lat_vec_top2 + 0.5*(2*sublattice_top-1)*tau
            dists.append([DistBottomTop(vec_bottom_atom,sublattice_bottom, vec_top_atom,sublattice_top),[n,k]])
    distances = ExtractFirsts(dists)
    index = np.argmin(np.array(distances))
    vec_top_atom = dists[index][1][0]*lat_vec_top1+dists[index][1][1]*lat_vec_top2 + 0.5*(2*sublattice_top-1)*tau
    delta = vec_top_atom-vec_bottom_atom
    return [dists[index],delta]

def ClosestAtomFromTopAimed(aim_atom, bottom_atom, sublattice_bottom, sublattice_top):
    
    # finds [n,k] of atom from top layer within given sublattice
    # which is the closest to given [i,j] atom from sublattice_bottom from the bottom
    # returns distance to closest atom, its coordinates and vector in plane which connects bottom atom to top atom
    # uses aim (closest atom calculated in previous step) and the window=4 for searching the next closest atom
    
    vec_bottom_atom = bottom_atom[0]*lat_vec_bottom1+bottom_atom[1]*lat_vec_bottom2 + 0.5*(2*sublattice_bottom-1)*tau
    dists = []
    window = 4 # a largest possible shift in indices when the previous closest atom is known
    for n in range(aim_atom[0]-window, aim_atom[0]+window+1):
        for k in range(aim_atom[1]-window, aim_atom[1]+window+1):
            vec_top_atom = n*lat_vec_top1+k*lat_vec_top2 + 0.5*(2*sublattice_top-1)*tau
            dists.append([DistBottomTop(vec_bottom_atom,sublattice_bottom, vec_top_atom,sublattice_top),[n,k]])
    distances = ExtractFirsts(dists)
    index = np.argmin(np.array(distances))
    vec_top_atom = dists[index][1][0]*lat_vec_top1+dists[index][1][1]*lat_vec_top2 + 0.5*(2*sublattice_top-1)*tau
    delta = vec_top_atom-vec_bottom_atom
    return [dists[index],delta]

#print(ClosestAtomFromTop([4,2],0,0))

Vppsigma = 0.48 # eV
Vpppi = -2.7 # eV
delta0 = 0.184*a # angstrom 
h0 = 3.349 # angstrom
rc = 6.14 # angstrom
lc = 0.265 # angstrom

def Hopping(vec, sublattice_bottom, sublattice_top):
    
    # vec is a vector connecting grid elements from bottom and top layer in 2D (top minus bottom)
    # returns value -t(d) following Moon Koshino (2013)
    
    vec = vec + (sublattice_top-sublattice_bottom)*tau
    full_dist = np.sqrt(np.linalg.norm(vec)**2+d0**2)
    Vpi = Vpppi*np.exp(-(full_dist-a0)/delta0)
    Vsigma = Vppsigma*np.exp(-(full_dist-d0)/delta0)
    return Vpi*(1-(d0/full_dist)**2)+Vsigma*(d0/full_dist)**2

#Hopping([-0,0],0,0)

G1M = 8*np.pi/(np.sqrt(3)*a)*np.sin(0.5*theta)*np.array([1,0])
G2M = 8*np.pi/(np.sqrt(3)*a)*np.sin(0.5*theta)*np.array([-0.5,0.5*np.sqrt(3)])

K = 2*np.pi/a*np.array([-2/3,0])
Ktheta = np.matmul(Rotation(theta), K)

def HamiltForK(bottom_atom, sublattice_bottom, sublattice_top):
    
    # bottom atom is an array [i,j]
    # tight-binding model with hoppings to 9 closest neighbours
    # returns value of hamiltonian matrix element between bloch states
    # with given wave number K from Dirac point

    top_atom = ClosestAtomFromTop(bottom_atom, sublattice_bottom, sublattice_top)
    delta = top_atom[1]
    ham = 0
    for i in np.arange(-1,2):
        for j in np.arange(-1,2):
            vec = delta + i*lat_vec_bottom1 + j*lat_vec_bottom2
#vec = delta + i*lat_vec_top1 + j*lat_vec_top2
            phase = -np.dot(K, vec)
            ham += Hopping(vec,0,0)*np.exp(1.j*phase) # delta knows about sublattices indices hence 0,0 in this formula
    return [ham, top_atom]


def HamiltForKAimed(aim_atom, bottom_atom, sublattice_bottom, sublattice_top):
    
    # bottom atom is an array [i,j]
    # tight-binding model with hoppings to 9 closest neighbours
    # returns value of hamiltonian matrix element between bloch states
    # with given wave number K from Dirac point

    top_atom = ClosestAtomFromTopAimed(aim_atom, bottom_atom, sublattice_bottom, sublattice_top)
    delta = top_atom[1]
    ham = 0
    for i in np.arange(-1,2):
        for j in np.arange(-1,2):
            vec = delta + i*lat_vec_bottom1 + j*lat_vec_bottom2
#vec = delta + i*lat_vec_top1 + j*lat_vec_top2
            phase = -np.dot(K, vec)
            ham += Hopping(vec,0,0)*np.exp(1.j*phase) # delta knows about sublattices indices hence 0,0 in this formula
    return [ham, top_atom]
        
#HamiltForK([0,5],0,0)

def FourierHopping(moire_vec, sublattice_bottom, sublattice_top):
    
    # moire vec is described by an array [n,k] and equals n*G1M+k*G2M
    # returns fourier amplitude of hopping function
    
    ampl = 0
    flag = 0

    for n in np.arange(-N, N+1):
        for k in np.arange(-N, N+1):
            if flag==0:
                vec_bottom_atom = n*lat_vec_bottom1+k*lat_vec_bottom2 
                hamiltonian = HamiltForK([n,k], sublattice_bottom, sublattice_top)
                hopping = hamiltonian[0]
                top_atom = hamiltonian[1]
                delta = top_atom[1]
                aim_atom = top_atom[0][1]
                G = moire_vec[0]*G1M + moire_vec[1]*G2M
                phase = -np.dot(G, vec_bottom_atom) - np.dot(K, delta) # phase before delta may be - or +
                ampl += hopping*np.exp(1.j*phase)
                flag = 1 
            else:
                vec_bottom_atom = n*lat_vec_bottom1+k*lat_vec_bottom2 
                hamiltonian = HamiltForKAimed(aim_atom, [n,k], sublattice_bottom, sublattice_top)
                hopping = hamiltonian[0]
                top_atom = hamiltonian[1]
                aim_atom = top_atom[0][1]
                G = moire_vec[0]*G1M + moire_vec[1]*G2M
                phase = np.dot(G, vec_bottom_atom)
                ampl += hopping*np.exp(1.j*phase)
    return np.abs(ampl)


print(theta, N, -1, -1 , FourierHopping([-1,-1],0,0))
print(theta, N, -1, 1 , FourierHopping([-1,1],0,0))
print(theta, N, 1, 0 , FourierHopping([1,0],0,0))
print(theta, N, 1, 1 , FourierHopping([1,1],0,0))
print(theta, N, 0, 0 , FourierHopping([0,0],0,0))
print(theta, N, 0, 1 , FourierHopping([0,1],0,0))
print(theta, N, -1, 0 , FourierHopping([-1,0],0,0))
print(theta, N, 0, -1 , FourierHopping([0,-1],0,0))
print(theta, N, 1, -1 , FourierHopping([1,-1],0,0))
print(theta, N, 0, -2 , FourierHopping([0,-2],0,0))
print(theta, N, 1, -2 , FourierHopping([1,-2],0,0))

0.017453292519943295 100 -1 -1 434.4063380243378
0.017453292519943295 100 -1 1 156.94974776975704
0.017453292519943295 100 1 0 139.00104129208486
0.017453292519943295 100 1 1 1767.028654173573
0.017453292519943295 100 0 0 1796.7466787629119
0.017453292519943295 100 0 1 1984.6362461721028
0.017453292519943295 100 -1 0 96.02558580540503


KeyboardInterrupt: 

In [32]:
print(theta, N, 0, 0 , FourierHopping([0,0],0,0))
print(theta, N, 0, 1 , FourierHopping([0,1],0,0))
print(theta, N, -1, 0 , FourierHopping([-1,0],0,0))
print(theta, N, 0, -1 , FourierHopping([0,-1],0,0))
print(theta, N, 1, -1 , FourierHopping([1,-1],0,0))
print(theta, N, 0, -2 , FourierHopping([0,-2],0,0))
print(theta, N, 1, -2 , FourierHopping([1,-2],0,0))

0.08726646259971647 100 0 0 130.26709376559037
0.08726646259971647 100 0 1 106.61560356807769
0.08726646259971647 100 -1 0 7.5514857231226955
0.08726646259971647 100 0 -1 0.7917078931808584
0.08726646259971647 100 1 -1 2.1501646979736955
0.08726646259971647 100 0 -2 4.754993015077789
0.08726646259971647 100 1 -2 3.0674729948951684


In [20]:
print(theta, N, -1, 1 , FourierHopping([-1,1],0,0))

0.08726646259971647 100 -1 1 2.4602756569220876


In [21]:
print(theta, N, 0, 0 , FourierHopping([0,0],0,0))

0.08726646259971647 100 0 0 1.192578460826559
